Run with

    ruby run.rb > sentiment_map.json

Code:

    require 'csv'
    require 'json'
    require 'sentimental'
    
    DIALOGUE_CSV_FILE = 'dialogue.csv'
    
    sentiment = Sentimental.new
    sentiment.load_defaults
    
    sentiment_map = {}
    
    # Initialize sentiment_map with unique values from Character column
    csv = CSV.table(DIALOGUE_CSV_FILE)
    csv[:character].each do |character_name|
      sentiment_map[character_name] = {}
    end
    
    # Now iterate through each row in the CSV
    0.upto(csv.length - 1) do |r|
      row = csv[r]
      speaker, line = row.fetch(:character), row.fetch(:line)
    
      # Look for anyone mentioned in this line
      mentioned_characters = line.gsub(/[\.,\?!]/, '').split(' ').select do |word|
        sentiment_map.keys.include? word
      end
    
      line_sentiment = sentiment.sentiment line
      line_score     = sentiment.score     line
    
      mentioned_characters.each do |mentioned_character|
        unless sentiment_map[speaker].key? mentioned_character
          sentiment_map[speaker][mentioned_character] = {
            positive: 0,
            neutral:  0,
            negative: 0,
            cumulative_sentiment: 0
          }
        end
    
        sentiment_map[speaker][mentioned_character][line_sentiment] += 1
        sentiment_map[speaker][mentioned_character][:cumulative_sentiment] += line_score
      end
    
    end
    
    # Show sentiment map
    puts JSON.pretty_generate(sentiment_map)




This will output a graph in JSON representing:

    {
      speaking_character_1: {
        mentioned_character_1: {
          ... sentiment mentions and cumulative score ...
        },
        mentioned_character_2: {
          .. sentiment mentions and cumulative score ...
        },
        ...
      },
      speaking_character_2: {
        ...
      },
      ...
    }